In [ ]:
import numpy as np
import matplotlib
import pandas as pd
import tensorflow as tf

dataset = pd.read_csv('/Users/wzhang/Downloads/lake_erie_habs_W4_6_8_9_12_13_16_2013-2020.xlsx - HAB_data (1).csv')

train = dataset.iloc[:743,:]
test = dataset.iloc[743:,:]
print(train)
print(test)
X_train = pd.DataFrame(train.iloc[:,4:15].values)
Y_train = train.iloc[:,16:].values
print(X_train.shape)
print(Y_train.shape)
X_test = pd.DataFrame(test.iloc[:,4:15].values)
Y_test = test.iloc[:,16:].values
print(X_test.shape)
print(Y_test.shape)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [ ]:
from keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization, GRU, LayerNormalization, MultiHeadAttention,TimeDistributed, Input, Flatten
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def create_rf():
    # Initialize Random Forest regressor
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model
    rf_model.fit(X_train, Y_train)
    return rf_model




In [ ]:
import lightgbm as lgb
def create_lightgbm():
    params = { 
    'objective': 'regression', 
    'metric': 'rmse', 
    'boosting_type': 'gbdt', 
    'num_leaves': 31, 
    'learning_rate': 0.05, 
    'feature_fraction': 0.9, 
    }
    train_data = lgb.Dataset(X_train,label=Y_train)
    num_round = 100
    bst = lgb.train(params, train_data, num_round) 
    from sklearn.metrics import mean_squared_error as mse 
    from lightgbm import LGBMRegressor 
    
    # Create an instance of the LightGBM Regressor with the RMSE metric. 
    model = LGBMRegressor(metric='rmse') 
    
    # Train the model using the training data. 
    model.fit(X_train, Y_train) 

    res = model.predict(X_test) 
    return model


In [ ]:
rf = create_rf()
lightgbm = create_lightgbm()

# ridge = create_ridge()
# svm = create_SVM()

In [ ]:
from tensorflow.keras.layers import concatenate


# gruPredict = gru.predict(X_train)
# lassoPredict = lasso.predict(X_train)
# lstmPredict = lstm.predict(X_train)
rfPredict = rf.predict(X_train)
lightgbmPredict = lightgbm.predict(X_train)
# ridgePredict = ridge.predict(X_train)
# svmPredict = svm.predict(X_train)
# rfPredict = rf.predict(X_train)
# lassoPredict.resize(671,1)
rfPredict.resize(743,1)
lightgbmPredict.resize(743,1)
# svmPredict.resize(671,1)
trainData = concatenate([rfPredict,lightgbmPredict])

In [ ]:
import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(trainData, Y_train, enable_categorical=True)

In [ ]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
)

In [ ]:
# gruPredictTest = gru.predict(X_test)
# lassoPredictTest = lasso.predict(X_test)
# lstmPredictTest = lstm.predict(X_test)
rfPredictTest = rf.predict(X_test)
lightgbmPredictTest = lightgbm.predict(X_test)
# ridgePredictTest = ridge.predict(X_test)
# svmPredictTest = svm.predict(X_test)
# svmPredictTest.resize(168,1)
# svmPredictTest.resize(96,1)
# lassoPredictTest.resize(168,1)
rfPredictTest.resize(96,1)
lightgbmPredictTest.resize(96,1)
testData = concatenate([rfPredictTest, lightgbmPredictTest])

In [ ]:
dtest_reg = xgb.DMatrix(testData, Y_test, enable_categorical=True)

In [ ]:
res = model.predict(dtest_reg)
print(res)

In [ ]:
from sklearn.metrics import r2_score 
r2 = r2_score(Y_test, res) 
print(r2)

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(Y_test,res)
print(mae)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(Y_test,res)
print(mape)

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(Y_test,res,squared=False)
print(rmse)